In [1]:
import pandas as pd
import re
import konlpy
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
from tqdm import tqdm
import urllib.request

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-2, 2))

In [2]:
# 입력 데이터프레임은 다음과 같은 양식으로 통일할 것
df = pd.read_table('최종네이버리뷰.txt', sep='/n')
df = df[['document']]
df = df.rename(columns={"document": "sentence"})
df

<ipython-input-2-3aa9ffd488a5>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_table('최종네이버리뷰.txt', sep='/n')


,sentence
0,0
1,"""선불이고 브레이크 타임은 2시 30분부터 5시까지이고 1인당 9900원에 9가지 ..."
2,가성비는 있는데 맛은 제 입에 안 맞았어요 탕수육은 맛있음 ??
3,음식 맛있어요사람들 정말 많아요다먹은후 그릇은셀프로 치워야해요^^짬뽕국물먹으러 또가...
4,정말 맛있게 잘먹었습니다.
...,...
12459,이문동에서 제일 인기많은 카페라고 생각이 드네요?? 분위기가 따스하고 안락하니 좋아요.
12460,음료가 맛있고 수플레가 진짜 맛있어요!
12461,전체적으로 아늑한 분위기예요 화장실은 아쉬워요
12462,수플레 첨먹어보는뎅 맛있어요 살살 녹아용ㅎㅎ 양귀비아이스티도 짱맛!


In [3]:
sent_dic = pd.read_csv('SentiWord_Dict.txt',sep = '\t',header=None)
sent_dic.iloc[14850,0]='갈등'

pos_dic = sent_dic[sent_dic[1]>0]
neg_dic = sent_dic[sent_dic[1]<0]
neu_dic = sent_dic[sent_dic[1]==0]

In [14]:
sent_dic[632:650]

,0,1
632,건강에 긍정적인,1
633,건강에 좋은,2
634,건강에 해로운,-2
635,건강하고,2
636,건강하고 온전하다,2
637,건강하고 온전한,2
638,건강하다,2
639,건강하여,2
640,건강하여 아름다운,2
641,건강하지 못하다,-1


In [4]:
okt = konlpy.tag.Okt()

def text_preprocess(x):
    text=[]
    a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x):
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
            text.append(token[0])
    return text

In [5]:

tqdm.pandas()
df['comment'] = df['sentence'].apply(lambda x : text_preprocess(x))
df['comment'] = df['comment'].progress_apply(lambda x: tokenize(x))
df

 18%|█████████████▊                                                              | 2266/12464 [00:11<00:52, 193.21it/s]


KeyboardInterrupt: 

In [ ]:

def make_sent_dict(x) :
    pos=[]
    neg=[]
    tmp={}
    
    for sentence in tqdm(x):
        for word in sentence :
            target = sent_dic[sent_dic[0]==word]
            if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                score = float(target[1])
                if score > 0:
                    pos.append(word)
                elif score < 0:
                    neg.append(word)                
            tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
    pos = list(set(pos))
    neg = list(set(neg))
    
    for sentence in tqdm(x):
        for word in sentence :
            tmp[word]['W'] += 1 # 빈도 수
            for po in pos :
                if po in sentence:
                    tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                    break
            for ne in neg:
                if ne in sentence:
                    tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                    break
    return pos, neg, pd.DataFrame(tmp)

In [ ]:

pos, neg, new_dict = make_sent_dict(df['comment'].values)
new_dict

In [ ]:

def make_score_dict(d,p,n):
    N=sum(d.iloc[0,::])
    pos_cnt=sum(d.loc[::,p].iloc[0,::])
    neg_cnt=sum(d.loc[::,n].iloc[0,::])
    
    trans =d.T
    trans['neg_cnt']=neg_cnt
    trans['pos_cnt']=pos_cnt
    trans['N']=N

    trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
    trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
    trans['SO_MI']=trans['MI_P'] - trans['MI_N']
    
    trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    trans = trans.sort_values(by=['SO_MI'],ascending=False)
    return trans

In [ ]:
t_dict = make_score_dict(new_dict,pos,neg)
t_dict['SO_MI'] = scaler.fit_transform(t_dict['SO_MI'].values.reshape(-1,1))
t_dict

In [ ]:

def update_dict(d):
    add_Dic = {0:[],1:[]}
    for i in d.T.items():
        if i[0] not in list(sent_dic[0]):
            if len(i[0]) > 1:
                add_Dic[0].append(i[0])
                add_Dic[1].append(i[1]['SO_MI'])
            
    add_Dic=pd.DataFrame(add_Dic)
    Sentiment=pd.merge(sent_dic,add_Dic,'outer')
    return Sentiment

In [ ]:
add_dict =update_dict(t_dict)
add_dict

In [ ]:

def get_cnt(x):
    cnt = 0
    for word in list(set(x)):
        target = add_dict[add_dict[0]==word]
        if len(target)==1:
            cnt += float(target[1])
    return cnt

def get_ratio(x):
    score = x['score']
    length = np.log10(len(x['comment']))+1
    try:
        ratio= round(score/length,2)
    except:
        ratio = 0
    return ratio

In [ ]:
tqdm.pandas()
df['score']= df['comment'].progress_apply(lambda x : get_cnt(x))
df['ratio'] = df.apply(lambda x: get_ratio(x), axis = 1)

In [ ]:
df

In [ ]:
class Aurora3:
    
    def __init__(self, df,sent_dic):
        self.df = df
        self.okt = konlpy.tag.Okt()
        self.sent_dic = sent_dic
        
    def get_df(self):# 최종 결과 반환
        print("문장을 토큰화 중입니다...")
        self.tokenizer_run()
        
        print("감성사전을 업데이트 중입니다...")
        self.expand_sent_dic()
        
        print("문장 감성분석 중입니다....")
        self.sent_analyze()
        return self.df
        
    def tokenizer_run(self): # 텍스트 전처리 & 토큰화
        tqdm.pandas()
        
        def text_preprocess(x):
            text=[]
            a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
            for j in a.split():
                text.append(j)
            return ' '.join(text)

        def tokenize(x):
            text = []
            tokens = self.okt.pos(x)
            for token in tokens :
                if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
                    text.append(token[0])
            return text
        self.df['comment'] = self.df['sentence'].apply(lambda x : text_preprocess(x))
        self.df['comment'] = self.df['comment'].progress_apply(lambda x: tokenize(x))
    
    def expand_sent_dic(self):
        sent_dic = self.sent_dic
        
        def make_sent_dict(x) :
            pos=[]
            neg=[]
            tmp={}

            for sentence in tqdm(x):
                for word in sentence :
                    target = sent_dic[sent_dic[0]==word]
                    if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                        score = float(target[1])
                        if score > 0:
                            pos.append(word)
                        elif score < 0:
                            neg.append(word)                
                    tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
            pos = list(set(pos))
            neg = list(set(neg))

            for sentence in tqdm(x):
                for word in sentence :
                    tmp[word]['W'] += 1 # 빈도 수
                    for po in pos :
                        if po in sentence:
                            tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                            break
                    for ne in neg:
                        if ne in sentence:
                            tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                            break
            return pos, neg, pd.DataFrame(tmp)
        
        def make_score_dict(d,p,n):
            N=sum(d.iloc[0,::])
            pos_cnt=sum(d.loc[::,p].iloc[0,::])
            neg_cnt=sum(d.loc[::,n].iloc[0,::])

            trans =d.T
            trans['neg_cnt']=neg_cnt
            trans['pos_cnt']=pos_cnt
            trans['N']=N

            trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
            trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
            trans['SO_MI']=trans['MI_P'] - trans['MI_N']

            trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
            trans = trans.sort_values(by=['SO_MI'],ascending=False)
            return trans
        
        def update_dict(d):
            add_Dic = {0:[],1:[]}
            for i in d.T.items():
                if i[0] not in list(sent_dic[0]):
                    if len(i[0]) > 1:
                        add_Dic[0].append(i[0])
                        add_Dic[1].append(i[1]['SO_MI'])

            add_Dic=pd.DataFrame(add_Dic)
            Sentiment=pd.merge(sent_dic,add_Dic,'outer')
            return Sentiment
        
        self.pos, self.neg, self.new_dict = make_sent_dict(self.df['comment'].values)
        
        self.t_dict = make_score_dict(self.new_dict,self.pos,self.neg)
        self.t_dict['SO_MI'] = scaler.fit_transform(self.t_dict['SO_MI'].values.reshape(-1,1))
       
        self.add_dict =update_dict(self.t_dict)
    
    def sent_analyze(self): # 데이터 감성분석
        tqdm.pandas()
        
        def get_cnt(x):
            cnt = 0
            for word in list(set(x)):
                target = self.add_dict[self.add_dict[0]==word]
                if len(target)==1:
                    cnt += float(target[1])
            return cnt

        def get_ratio(x):
            score = x['score']
            length = np.log10(len(x['comment']))+1
            try:
                ratio= round(score/length,2)
            except:
                ratio = 0
            return ratio
        
        tqdm.pandas()
        self.df['score']= self.df['comment'].progress_apply(lambda x : get_cnt(x))
        self.df['ratio'] = self.df.apply(lambda x: get_ratio(x), axis = 1)

In [ ]:
test = Aurora3(df,sent_dic)
res = test.get_df()

In [ ]:
res

In [ ]:
res[458:477]

In [ ]:
#res.to_csv('네이버리뷰감성점수.csv', encoding='cp949',index=False)